In [ ]:
import os

In [ ]:
os.chdir('../')

In [ ]:
import FeedForwardANN as FFANN
import numpy as np

%matplotlib notebook
# %matplotlib widget
import matplotlib.pyplot as plt


plt.rcParams['font.size']=16
plt.rcParams['font.family']='dejavu sans'

plt.rcParams['mathtext.fontset']='stix'
plt.rcParams['mathtext.rm']='custom'
plt.rcParams['mathtext.it']='stix:italic'
plt.rcParams['mathtext.bf']='stix:bold'

In [ ]:
os.chdir('example')

In [ ]:
lin=FFANN.linearActivation()
sig=FFANN.sigmoidActivation()
exp=FFANN.expActivation()



In [ ]:
#the squared error and its derivative
def Q_i(model, i, target):
    return (model.signals[model.total_layers-1][i]-target)**2


def dQds_i(model, i, target):
    return 2*(model.signals[model.total_layers-1][i]-target)


In [ ]:
_=np.loadtxt('eos2020.dat')
T=np.array([[np.log10(i[0])] for i in _[:,] ])
rdof=np.array([[i[1],i[2]] for i in _[:]])
# rdof=np.array([[np.log(i[1]),np.log(i[2])] for i in _[1:]])

In [ ]:
brain=FFANN.FFANN(1,2,[4],[sig,lin])

brain.init_params(-1e-1,1e-1)



Q=FFANN.loss(Q_i, dQds_i, brain)



# strategy=FFANN.VanillaSGD(brain,Q,alpha=1e-1)
# strategy=FFANN.RMSpropSGD(brain,Q,gamma=0.999,epsilon=1e-5,alpha=1e-2)
# strategy=FFANN.AdaDeltaSGD(brain,Q,gamma=0.995,epsilon=1e-4,alpha=1)
# strategy=FFANN.AdamSGD(brain,Q,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-1)
strategy=FFANN.AdaMaxSGD(brain,Q,beta_m=1-1e-2,beta_v=1-1e-4,epsilon=1e-8,alpha=1e-1)
# strategy=FFANN.AdamSGD(brain,Q,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-1)


In [ ]:
brain.SGD(strategy, T, rdof, abs_tol=1e-4, rel_tol=1e-4, step_break=1000,max_step=150000)

In [ ]:
brain.inputSignal([5])
brain.feedForward()    
brain.signals[-1]
brain.signals[-1],rdof[-1]

In [ ]:
brain.inputSignal([-5])
brain.feedForward()
brain.signals[-1],rdof[0]

In [ ]:
fig=plt.figure(figsize=(9,5))
fig.subplots_adjust(bottom=0.15, left=0.15, top = 0.9, right=0.9,wspace=0.0,hspace=0.0)

sub = fig.add_subplot(1,1,1)

g=[]
for logT in T:
    brain.inputSignal(logT)
    brain.feedForward()    
    g.append(brain.signals[-1][1]) 
g=np.array(g)
sub.plot(T,g,linestyle='-',linewidth=3.5,alpha=1,c='xkcd:black',label=r'FFANN')
sub.plot(T,rdof[:,1],linestyle='--',linewidth=2.5,alpha=1,c='xkcd:red',label=r'data')


sub.legend(bbox_to_anchor=(0.01, 0.99),borderaxespad=0., columnspacing=1,labelspacing=-0.1,
           borderpad=0,ncol=1,loc='upper left' ,framealpha=0)



sub.set_xlabel(r'$log_{10}\left( \dfrac{T}{{\rm GeV}} \right)$')
sub.xaxis.set_label_coords(0.5, -0.075) 
sub.set_ylabel(r'$g_{\rm *}$')
sub.yaxis.set_label_coords(-0.07,0.5) 


fig.show()

In [ ]:
fig=plt.figure(figsize=(9,5))
fig.subplots_adjust(bottom=0.15, left=0.15, top = 0.9, right=0.9,wspace=0.0,hspace=0.0)

sub = fig.add_subplot(1,1,1)

h=[]
for logT in T:
    brain.inputSignal(logT)
    brain.feedForward()    
    h.append(brain.signals[-1][0])
    
h=np.array(h)
sub.plot(T,h,linestyle='-',linewidth=3.5,alpha=1,c='xkcd:black',label=r'FFANN')
sub.plot(T,rdof[:,0],linestyle='--',linewidth=2.5,alpha=1,c='xkcd:red',label=r'data')


sub.legend(bbox_to_anchor=(0.01, 0.99),borderaxespad=0., columnspacing=1,labelspacing=-0.1,
           borderpad=0,ncol=1,loc='upper left' ,framealpha=0)



sub.set_xlabel(r'$log_{10}\left( \dfrac{T}{{\rm GeV}} \right)$')
sub.xaxis.set_label_coords(0.5, -0.075) 
sub.set_ylabel(r'$h_{\rm eff}$')
sub.yaxis.set_label_coords(-0.07,0.5) 


fig.show()

In [ ]:
fig=plt.figure(figsize=(9,5))
fig.subplots_adjust(bottom=0.15, left=0.15, top = 0.9, right=0.9,wspace=0.0,hspace=0.0)

sub = fig.add_subplot(1,1,1)

    
sub.plot(T,np.abs(g-rdof[:,1])/rdof[:,1],linestyle='-',linewidth=2.5,alpha=1,c='xkcd:black',label=r'$g_{\rm *}$')
sub.plot(T,np.abs(h-rdof[:,0])/rdof[:,1],linestyle='--',linewidth=2.5,alpha=1,c='xkcd:red',label=r'$h_{\rm eff}$')


sub.legend(bbox_to_anchor=(0.01, 0.99),borderaxespad=0., columnspacing=1,labelspacing=-0.1,
           borderpad=0,ncol=1,loc='upper left' ,framealpha=0)



sub.set_xlabel(r'$log_{10}\left( \dfrac{T}{{\rm GeV}} \right)$')
sub.xaxis.set_label_coords(0.5, -0.075) 
sub.set_ylabel(r'relative difference')
sub.yaxis.set_label_coords(-0.1,0.5) 


fig.show()